<a href="https://colab.research.google.com/github/Arvinzaheri/Recommender-Systems-with-pytorch/blob/main/what_to_buy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1 dir=rtl align=center style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
چی بخریم؟
</font>
</h1>


<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
به یک پروژه‌ی متفاوت و کاربردی خوش آمدید! 🤩 در این مسئله گریزی به <i>سیستم‌های پیشنهاددهنده (Recommender Systems)</i> خواهیم زد و با استفاده از یک معماری یادگیری عمیق خلاقانه، مدلی پیاده خواهیم کرد که بتواند علاقه‌ی یک کاربر به یک کالای مشخص را تخمین بزند. با استفاده از این مدل خواهیم توانست لیستی از کالاها را به کاربر پیشنهاد دهیم که تاکنون آن‌ها را خریداری نکرده اما طبق الگوهایی که مدل بین کاربران و کالاها پیدا کرده به‌احتمال زیاد از آن‌ها خوشش خواهد آمد و به آن‌ها امتیاز بالایی خواهد داد. مجموعه‌داده‌ی پروژه نیز شامل داده‌های واقعی مربوط به کالاهای الکترونیکی موجود در <i>فروشگاه آنلاین آمازون (Amazon)</i> است.
</font>
</p>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font color="orange"><b>نکته:</b></font>
<font face="vazir" size=3>
اگرچه در این پروژه با داده‌های فروشگاه آنلاین سروکار داریم اما این مدل پیشنهاددهنده را می‌توان در دامنه‌های گوناگونی به کار گرفت. به‌عنوان مثال می‌توان سیستمی مشابه جهت پیشنهاد فیلم یا قطعه‌ی موسیقی به کاربران طراحی کرد.
</font>
</p>

<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
مجموعه‌داده
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
داده‌های آموزشی در فایل <code dir=ltr>amazon_train.csv</code> در دسترس شما قرار گرفته است. این فایل شامل ۴ ستون است که به‌ترتیب بیانگر شناسه‌ی کاربر، شناسه‌ی محصول، امتیازی که کاربر به محصول داده و زمان ثبت امتیاز هستند. شبکه‌ای که در ادامه پیاده خواهیم کرد شناسه‌ی عددی کاربر و محصول را ورودی می‌گرد و امتیازی که کاربر به آن محصول می‌دهد را تخمین خواهد زد. ابتدا فایل مجموعه‌داده را به‌کمک <code>pandas</code> بخوانید.
</font>
</p>

In [7]:
import numpy as np
import pandas as pd

# Loading the dataset
# TODO
train = pd.read_csv("amazon_train.csv")
valid = pd.read_csv("amazon_valid.csv")
test = pd.read_csv("amazon_test.csv")

print(f"train shape: {train.shape}")
print(f"valid shape: {valid.shape}")
print(f"test shape: {test.shape}")

train shape: (909201, 4)
valid shape: (89919, 5)
test shape: (89732, 4)


<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
پیش‌پردازش
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
پس از بررسی و تحلیل داده‌هایی که در اختیارتان گرفته ممکن است به پیش‌پردازش‌های مختلفی نیاز پیدا کنید و ایده‌های گوناگونی را پیاده کنید. با این حال، دو پیش‌پردازش اصلی که حتماً به آن‌ها نیاز خواهید داشت را در ادامه شرح داده‌ایم.
</font>
</p>

<h3 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
کدگذاری
</font>
</h3>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
از آن‌جا که شناسه‌های کاربر (<code dir=ltr>UserID</code>) و محصول (<code dir=ltr>ProductID</code>) به‌صورت رشته‌ای هستند، نیاز است آن‌ها را به‌شکل اعداد مرتب کدگذاری کنیم. بنابراین ابتدا با استفاده از کلاس <code>LabelEncoder</code> موجود در کتابخانه‌ی <code>sklearn</code> هر کدام از این ستون‌ها را کدگذاری کنید. پیشنهاد می‌شود حاصل را در ستون‌های مجزایی به‌نام‌های <code>UserEnc</code> و <code>ProductEnc</code> ذخیره کنید.
</font>
</p>

In [8]:
train.head()

,UserID,ProductID,Rating,Timestamp
0,A3HICVLF4PFFMN,0594481813,5.0,2014-05-05
1,A2QBZA4S1ROX9Q,0594481813,3.0,2013-05-25
2,AT09WGFUM934H,0594481813,3.0,2013-08-31
3,AGAKHE014LQFU,0594481813,3.0,2013-09-18
4,A1S6B5QFWGVL5U,0594481813,4.0,2013-06-27


In [9]:
train.isna().sum()

UserID       0
ProductID    0
Rating       0
Timestamp    0
dtype: int64

In [10]:
train.describe()

,Rating
count,909201.000000
mean,4.247838
std,1.151556
min,1.000000
25%,4.000000
50%,5.000000
75%,5.000000
max,5.000000


In [11]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
columns = ["UserID", "ProductID"]
for col in columns:
    train[col] = encoder.fit_transform(train[col])
    valid[col] = encoder.transform(valid[col])
    test[col] = encoder.transform(test[col])
# Encoding each ID string
# TODO
train.head()

,UserID,ProductID,Rating,Timestamp
0,59277,0,5.0,2014-05-05
1,41167,0,3.0,2013-05-25
2,85549,0,3.0,2013-08-31
3,77047,0,3.0,2013-09-18
4,18672,0,4.0,2013-06-27


<h3 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
نرمال‌سازی
</font>
</h3>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
اکنون مقادیر ستون <code>Rating</code> را به‌کمک <code>MinMaxScaler</code> کتابخانه‌ی <code>sklearn</code> نرمال کنید تا مقادیر آن به بازه‌ی ۰ تا ۱ نگاشت شود.
</font>
</p>

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
train["Rating"] = scaler.fit_transform(train["Rating"].values.reshape(-1, 1))
valid["Rating"] = scaler.transform(valid["Rating"].values.reshape(-1, 1))
# Scaling the Rating column
# TODO

In [ ]:
valid.head()

In [12]:
#valid.drop(columns=["UserEnc", "Timestamp"], inplace=True)
#train.drop(columns=["Timestamp"], inplace=True)
#test.drop(columns=["Timestamp"], inplace=True)
valid.drop(columns=["UserEnc"], inplace=True)

In [ ]:
train["Timestamp"] = pd.to_datetime(train["Timestamp"])
valid["Timestamp"] = pd.to_datetime(valid["Timestamp"])
test["Timestamp"] = pd.to_datetime(test["Timestamp"])

In [ ]:
train["year"] = train["Timestamp"].dt.year
train["month"] = train["Timestamp"].dt.month
train.drop(columns=["Timestamp"], inplace=True)

valid["year"] = valid["Timestamp"].dt.year
valid["month"] = valid["Timestamp"].dt.month
valid.drop(columns=["Timestamp"], inplace=True)

test["year"] = test["Timestamp"].dt.year
test["month"] = test["Timestamp"].dt.month
test.drop(columns=["Timestamp"], inplace=True)



In [ ]:
from sklearn.preprocessing import StandardScaler
scaler_2 = StandardScaler()
time_cols = ["year", "month"]
for col in time_cols:
    train[col] = scaler_2.fit_transform(train[col].values.reshape(-1, 1))
    valid[col] = scaler_2.transform(valid[col].values.reshape(-1, 1))
    test[col] = scaler_2.transform(test[col].values.reshape(-1, 1))

In [ ]:
train.head()

<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
مدل‌سازی
</font>
</h2>


<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
حال نوبت به مدل‌سازی می‌رسد. ساختار پیشنهادی ما به این شکل است که شبکه دو ورودی خواهد داشت: شناسه‌ی کاربر و شناسه‌ی کالا. برای هر کدام از این شناسه‌ها یک لایه‌ی تعبیه‌ساز (<code>Embedding</code>) جداگانه در نظر می‌گیریم تا به‌ازای هر کاربر یا کالا، یک بردار تعبیه‌ی بامعنا ساخته شود. خروجی هر لایه‌ی تعبیه به‌کمک یک لایه‌ی <code>Flatten</code> مسطح خواهد شد. پس از این مراحل، دو بردار بازنمایی کالا و کاربر در اختیار شما قرار می‌گیرد و می‌توانید پس از آن معماری دلخواه خود را پیاده کنید. به‌عنوان مثال می‌توانید بردار تعبیه‌ی کالا را در بردار تعبیه‌ی کاربر ضرب نقطه‌ای کنید (به‌کمک لایه‌ی <code>dot</code>) و حاصل را از تابع فعال‌ساز <code>sigmoid</code> عبور دهید تا خروجی شبکه یعنی امتیاز کاربر به آن کالا بین ۰ تا ۱ تولید شود. بنابراین ابتدا این ساختار را به‌کمک مدل تابعی (Functional) کراس پیاده‌سازی کنید.
</font>
</p>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font color="orange"><b>نکته:</b></font>
<font face="vazir" size=3>
دست شما کاملاً برای ایجاد تغییرات در این ساختار باز است. به‌عنوان مثال می‌توانید علاوه بر تولید بردارهای تعبیه، به‌کمک لایه‌های اضافه‌ی <code>Embedding</code> (با اندازه‌ی بردار ۱) یک مقدار بایاس هم برای هر کاربر یا کالا تولید کنید. در نهایت می‌توانید بایاس کاربر، بایاس کالا و حاصل ضرب نقطه‌ای را به‌کمک یک لایه‌ی <code>add</code> جمع کنید. به‌عنوان ایده‌ای دیگر می‌توانید به‌جای آن‌که دو بردار تعبیه را در یکدیگر ضرب کنید از عملیات‌های دیگری همچون الحاق (<code>Concatenate</code>) استفاده کنید. همچنین می‌توانید بر تعداد لایه‌ها بی‌افزایید، یا روش مقداردهی اولیه‌ی وزن‌ها یا رگولاریزیشن هر لایه را تنظیم کنید.
</font>
</p>

In [20]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [58]:

import torch
import torch.nn as nn
import torch.nn.functional as F

class recomender_system(nn.Module):
    def __init__(self, num_users, num_items, embedding_dim):
        super(recomender_system, self).__init__()
        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        self.item_embedding = nn.Embedding(num_items, embedding_dim)
        self.user_bias = nn.Embedding(num_users, 1)
        self.item_bias = nn.Embedding(num_items, 1)

    def forward(self, user_ids, item_ids):
        user_embedded = self.user_embedding(user_ids.long())
        item_embedded = self.item_embedding(item_ids.long())
        user_bias = self.user_bias(user_ids.long()).squeeze()
        item_bias = self.item_bias(item_ids.long()).squeeze()
        interaction = (user_embedded * item_embedded).sum(1)
        output = torch.sigmoid(interaction)

        return 4 * output + 1

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
پس از طراحی معماری شبکه، می‌توانید آن را با ورودی دادن یک لیست شامل دو لیست شناسه‌ی کاربر و شناسه‌ی کالا آموزش دهید. متغیر هدف نیز همان ستون <code>Rating</code> خواهد بود. همچنین در صورتی‌که در لایه‌ی آخر از تابع فعال‌ساز <code>sigmoid</code> برای نورون خروجی استفاده کرده باشید می‌توانید از تابع زیان <code>binary_crossentropy</code> بهره ببرید. در صورت علاقه و نیاز می‌توانید از هر تکنیک دلخواهی در فرآیند آموزش مدل خود استفاده کنید.
</font>
</p>

In [30]:
from sklearn.metrics import mean_absolute_percentage_error as mape

In [59]:
num_users = train["UserID"].nunique()
num_items = train["ProductID"].nunique()
embedding_dim = 50

Model = recomender_system(num_users, num_items, embedding_dim).to(device)

In [60]:
# Training phase
# TODO
learning_rate = 0.001
batch_size = 64
epochs = 10

loss_fn = nn.MSELoss()

optimizer = torch.optim.Adam(Model.parameters(), lr=learning_rate)

def metric(y_pred, y_true):

    return mape(y_true.cpu(), y_pred.detach().cpu().numpy())

In [38]:
from torch.utils.data import DataLoader, TensorDataset

# Convert DataFrame to PyTorch tensor
train_tensor= torch.tensor(train[["UserID", "ProductID", "Rating"]].values)
valid_tensor = torch.tensor(valid[["UserID", "ProductID", "Rating"]].values)
test_tensor = torch.tensor(test[["UserID", "ProductID"]].values)


train_dataset = TensorDataset(train_tensor[:, 0], train_tensor[:, 1], train_tensor[:, -1])  # Separate features, target, and other columns
valid_dataset = TensorDataset(valid_tensor[:, 0], valid_tensor[:, 1], valid_tensor[:, -1])  # Separate features, target, and other columns
test_dataset = TensorDataset(test_tensor[:, 0], test_tensor[:, 1])  # No target in the test set, include other columns

# Create DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)  # Create a dataloader for batching
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)  # Create a dataloader for batching
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)  # Create a dataloader for batching

In [61]:
def train_func(optimizer, train_batch):
    Model.train()
    users, items, ratings = train_batch
    users = users.to(device)
    items = items.to(device)
    ratings = ratings.to(device)
    ratings_pred = Model(users, items)
    loss = loss_fn(ratings_pred, ratings.float())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    pr = metric(ratings_pred, ratings.float())
    return loss, pr

def eval_func(valid_batch):
    users, items, ratings = valid_batch
    users = users.to(device)
    items = items.to(device)
    ratings = ratings.to(device)
    Model.eval()
    with torch.no_grad():
        ratings_pred = Model(users, items)
        loss = loss_fn(ratings_pred, ratings.float())
        pr = metric(ratings_pred, ratings.float())
    return loss, pr

In [62]:
from tqdm import tqdm
def train_loop(train, valid):
    for epoch in range(epochs):
        t_loss = []
        v_loss = []
        t_pr = []
        v_pr = []
        for i, batch in enumerate(tqdm(zip(train, valid))):
            train_batch, valid_batch = batch
            #print(train_batch)

            train_loss, train_pr = train_func(optimizer, train_batch)
            valid_loss, valid_pr = eval_func(valid_batch)

            t_loss.append(train_loss)
            v_loss.append(valid_loss)
            t_pr.append(valid_pr)
            v_pr.append(valid_pr)


        print(f"Epoch: {epoch}, Train Loss: {sum(t_loss) / len(t_loss)}, Valid Loss: {sum(v_loss) / len(v_loss)} Train PR: {sum(t_pr) / len(t_pr)}, Valid PR: {sum(v_pr) / len(v_pr)}")

In [56]:
epochs = 5

In [63]:
train_loop(train_dataloader, valid_dataloader)

1405it [00:09, 150.97it/s]


Epoch: 0, Train Loss: 6.089577674865723, Valid Loss: 6.058746814727783 Train PR: 0.5415461526646733, Valid PR: 0.5415461526646733


1405it [00:08, 159.52it/s]


Epoch: 1, Train Loss: 5.320267677307129, Valid Loss: 6.057563781738281 Train PR: 0.5415033063430379, Valid PR: 0.5415033063430379


1405it [00:08, 156.50it/s]


...

Epoch: 6, Train Loss: 2.3894662857055664, Valid Loss: 6.056120872497559 Train PR: 0.5416223257037669, Valid PR: 0.5416223257037669


1405it [00:09, 153.14it/s]


Epoch: 7, Train Loss: 2.112438678741455, Valid Loss: 6.055975437164307 Train PR: 0.541627286591988, Valid PR: 0.541627286591988


686it [00:04, 162.34it/s]


KeyboardInterrupt: 

<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
پیش‌بینی
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
دو فایل دیگر به نام‌های <code>amazon_valid.csv</code> و <code>amazon_test.csv</code> نیز به‌عنوان مجموعه‌ی اعتبارسنجی و آزمون در اختیار شما قرار گرفته است. مدل شما باید بتواند به‌ازای هر جفت کاربر و کالا، امتیازی که آن کاربر به آن کالا می‌دهد را به‌درستی تخمین بزند. ساختار این مجموعه‌ها نیز مشابه با مجموعه‌ی آموزشی است با این تفاوت که برچسب حقیقی داده‌های آزمون در دسترس شما نیست و ارزیابی مدل شما بر روی داده‌های آزمون در سامانه‌ی داوری کوئرا انجام می‌پذیرد. با این حال، توزیع داده‌ها بین مجموعه‌ی اعتبارسنجی و آزمون بسیار مشابه است و اگر مدل شما برای نمونه‌های اعتبارسنجی به عملکرد قابل قبولی برسد به احتمال زیاد برای نمونه‌های آزمون نیز همین‌گونه خواهد بود.
</font>
</p>


<h3 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
معیار ارزیابی
</font>
</h3>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
هدفی که در این نوع مسئله در نظر گرفته‌ایم این است که مدل شما تا چه‌اندازه می‌تواند کالاهایی که کاربر به آن‌ها امتیاز بالا (۴ یا ۵) می‌دهد را به‌درستی تشخیص دهد، بنابراین
عملکرد مدل شما در سامانه‌ی داوری به این‌گونه ارزیابی خواهد شد که برچسب‌های <code>5</code> و <code>4</code> به‌عنوان دسته‌ی مثبت و سایر برچسب‌ها به‌عنوان دسته‌ی منفی در نظر گرفته می‌شود.
سپس از معیار <code>Precision</code> جهت ارزیابی مدل شما استفاده خواهد شد که در واقع نسبت تعداد پیشنهادات مثبت درست (مدل تشخیص داده که کاربر به کالا امتیاز بالا می‌دهد و واقعاً این‌گونه بوده) به کل پیشنهادات مثبت مدل را می‌شمارد.
</font>
</p>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font color="red"><b color='red'>توجه:</b></font>
<font face="vazir" size=3>
 جهت کسب امتیاز کامل نیاز است تا پاسخ شما حداقل امتیاز <code>83</code> را با توجه به معیار معرفی‌شده کسب نماید.
</font>
</p>

In [ ]:
# Evaluate on validation data
# TODO

In [83]:
def predict(test_loader):
    Model.eval()
    all_predictions = []
    for batch in test_loader:
        users, items = batch[:2]  # فرض بر این است که دیتالودر دو عنصر برمی‌گرداند
        users = users.to(device)
        items = items.to(device)
        with torch.no_grad():
            predictions = Model(users, items).cpu().numpy()
            all_predictions.append(predictions)
    return np.concatenate(all_predictions)

<h3 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
ساختار خروجی برای نمونه‌های آزمون
</font>
</h3>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
 پیش‌بینی مدل خود را برای نمونه‌های آزمون به‌شکل امتیازی بین ۱ تا ۵ تولید کرده و در ستون <code>Rating</code> یک دیتافریم با نام <code>submission</code> در قالب جدول زیر ذخیره کنید.
</font>
</p>

<center>
<div dir=rtl style="direction: rtl;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
    
|ستون|توضیحات|
|:------:|:---:|
|<code>UserID</code>|شناسه‌ی کاربر (مشابه با نمونه‌ی آزمون)|
|<code>ProductID</code>|شناسه‌ی کالا (مشابه با نمونه‌ی آزمون)|
|<code>Rating</code>|امتیاز کاربر به کالا از ۱ تا ۵ (پیش‌بینی مدل)|
    
</font>
</div>
</center>

In [84]:
# Preparing submission file
submission = test[["UserID", "ProductID"]].copy()
submission["Rating"] = predict(test_dataloader)

In [85]:
submission.head()

,UserID,ProductID,Rating
0,46264,48174,1.105088
1,26515,48748,1.000047
2,66384,14729,3.948984
3,17487,48608,1.118785
4,4351,48714,4.986773


<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
<b>سلول جواب‌ساز</b>
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    برای ساخته‌شدن فایل <code>result.zip</code> سلول زیر را اجرا کنید. توجه داشته باشید که پیش از اجرای سلول زیر تغییرات اعمال شده در نت‌بوک را ذخیره کرده باشید (<code>ctrl+s</code>) تا در صورت نیاز به پشتیبانی امکان بررسی کد شما وجود داشته باشد.</font>
</p>

In [86]:
import zipfile
import joblib

def compress(file_names):
    print("File Paths:")
    print(file_names)
    compression = zipfile.ZIP_DEFLATED
    with zipfile.ZipFile("result.zip", mode="w") as zf:
        for file_name in file_names:
            zf.write('./' + file_name, file_name, compress_type=compression)

submission.to_csv('submission.csv', index=False)
file_names = ['what_to_buy.ipynb', 'submission.csv']
compress(file_names)

File Paths:
['what_to_buy.ipynb', 'submission.csv']


FileNotFoundError: [Errno 2] No such file or directory: './what_to_buy.ipynb'

<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
بیشتر بدانید:
سیستم پیشنهاددهنده
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
برای پیشنهاد کالای جدید به یک کاربر روش‌های مختلفی وجود دارد، اما یک راه ساده این است که از تخمین امتیازی که مدل به‌ازای هر جفت کاربر و کالا می‌دهد استفاده کنیم.
به‌ازای یک کاربر به‌کمک مدل خود تخمین می‌زنیم که وی از بین کالاهایی که تاکنون خریداری نکرده، کدام‌ها را بیشتر از همه دوست خواهد داشت و امتیاز بالاتری به آن‌ها خواهد داد. بنابراین می‌توانیم این کالاها را به او پیشنهاد دهیم و به‌احتمال زیاد از تعدادی از آن‌ها خوشش خواهد آمد. البته این موضوع نمی‌تواند مستقیماً به این معنی باشد که آن کالا را خریداری می‌کند. به‌عنوان مثال شاید یک کالای گران‌قیمت را پیشنهاد دهیم که همگی از آن خوشمان می‌آید اما بسیاری قدرت خرید آن را نداریم! 🫠
</font>
</p>


<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
<span style="color:green"><b>راهنمایي:</b></span>
برای تولید پیش‌بینی به‌ازای یک کاربر ابتدا لیستی از کالاهایی که وی تاکنون خریداری نکرده را پیدا کنید. سپس جفت شناسه‌ی کدگذاری‌شده‌ی کاربر و هر یک از این کالاها را به مدل ورودی بدهید تا امتیاز کاربر به هر کدام از آن کالاها تخمین زده شود. در نهایت می‌توان <code>n</code> کالایی که منجر به بیشترین امتیاز (خروجی مدل) شده را انتخاب کرد و به وی پیشنهاد داد.
</font>
</p>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
<span style="color:purple"><b>🧐 بیشتر بی‌اندیشیم:</b></span>
چه روش‌های دیگری جهت پیشنهاد کالا به یک کاربر مشخص به ذهن‌تان می‌رسد؟ آیا می‌توان مستقیماً‌ از بردارهای تعبیه‌ی هر کالا یا کاربر جهت پیشنهاد استفاده کرد؟ به چه روش‌های می‌توان کاربران مشابه با یک کاربر را پیدا کرده و براساس تجربه یا الگوی خرید آن‌ها به کاربر کالاهایی را پیشنهاد داد؟ پیشنهاد می‌کنیم به این موضوعات بیشتر فکر کرده و نظرات خود را از طریق بخش دیدگاه عمومی صفحه‌ی پروژه در کوئرا با دیگران به‌اشتراک بگذارید. چه بهتر که ایده‌های خود را به‌شکل عملی نیز آزمایش کنید.
</font>
</p>